In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install mxnet
# !pip install gluonnlp pandas tqdm
# !pip install sentencepiece
# !pip install transformers==3 # 최신 버전으로 설치하면 "Input: must be Tensor, not str" 라는 에러 발생
# !pip install torch
# !pip install torchmetrics

!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-d8bdpq41
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-d8bdpq41
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 KB 21.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 881.9/881.9 MB 56.6 MB/s eta 0:00

In [ ]:
# gpu 프리미엄 사용 시
!pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 2.0/2.0 GB 179.4 MB/s eta 0:00:01tcmalloc: large alloc 2041339904 bytes == 0x324a000 @  0x7f7ca18461e7 0x4d30a0 0x4d312c 0x5d6f4c 0x51edd1 0x51ef5b 0x4f750a 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x5d8868 0x4997a2 0x55cd91 0x5d8941 0x49abe4 0x55cd91 0x5d8941 0x4997a2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 2.0/2.0 GB 188.7 MB/s eta 0:00:01tcmalloc: large alloc 2551676928 bytes == 0x7cd10000 @  0x7f7ca1847615 0x5d6f4c 0x51edd1 0x51ef5b 0x4f750a 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x5d8868 0x4997a2 0x55cd91 0x5d8941 0x49abe4 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from sklearn.model_selection import train_test_split

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
import pandas as pd
from tqdm.notebook import tqdm

import re

In [ ]:
##GPU 사용 시
device = torch.device("cuda:0")

bertmodel, vocab = get_pytorch_kobert_model()

from transformers import AutoTokenizer, AutoModelForSequenceClassification


/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


### 데이터 전처리


In [ ]:
import pandas as pd

# 데이터 입력


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/뉴스원인분석_modeling/data/kia_text_rank_max_kss.csv')
df

,date_cate,content,전일비_비율,text_rank_content,len,text_rank_len,target,mpc_pos,mpc_neg,mpck_token,score_diff,score_diff+tar,pos_neg,pos_neg_tardiff,kmm_label,kmm_label_n5,kmm_label_n_n5
0,2017-12-04,현대기아차 북미시장 고전 지속코나스팅어 기대로스앤젤레스연합뉴스 옥철 특파원 현대차...,-0.15,"현대차는 오토쇼에 데뷔한 콤팩트 스포츠유틸리티차 코나ona, 기아차는 2018 ...",958,288,2,9.960439e-01,0.003956,현대기아차 북미시장 고전 지속코나스팅어 기대로스앤젤레스연합뉴스 옥철 특파원 현대차...,0.992088,2.492088,pos,pos,2,매우상승,4
1,2017-12-04,기업공시12월 1일서울경제 대표이사 손봉영에서 김종호로 변경 인터플렉스 유상증자에...,-0.15,기업공시12월 1일서울경제 대표이사 손봉영에서 김종호로 변경 인터플렉스 유상증자에...,1372,473,2,3.168241e-06,0.999997,기업공시12월 1일서울경제 대표이사 손봉영에서 김종호로 변경 인터플렉스 유상증자에...,-0.999994,0.500006,neg,pos,0,하락,1
2,2017-12-04,뒷좌석 어린이 안전도 평가카시트 장착한 6세 아동 충격 더 크다헤럴드경제박도제 기자...,-0.15,뒷좌석 어린이 안전도 평가카시트 장착한 6세 아동 충격 더 크다헤럴드경제박도제 기자...,1618,392,2,1.344403e-05,0.999987,뒷좌석 어린이 안전도 평가카시트 장착한 6세 아동 충격 더 크다헤럴드경제박도제 기자...,-0.999973,0.500027,neg,pos,0,하락,1
3,2017-12-04,"국산차, 연말 파격 할인싼타페 230만원캡티바 500만원이데일리 김보경 기자 국내 ...",-0.15,한국은 12월 한 달 동안 전 차종에 대해 515의 할인을 적용한다. 주력 차...,1014,237,2,3.493616e-05,0.999965,"국산차, 연말 파격 할인싼타페 230만원캡티바 500만원이데일리 김보경 기자 국내 ...",-0.999930,0.500070,neg,pos,0,하락,1
4,2017-12-04,상장 대목이번주 진에어 등 7개 증시 입성서울연합인포맥스 김경림 기자 이번 주4일...,-0.15,진에어는 공모청약 경쟁률 134. 동양피스톤은 내연기관용 피스톤을 만드는 회사로 ...,722,133,2,1.425929e-01,0.857407,상장 대목이번주 진에어 등 7개 증시 입성서울연합인포맥스 김경림 기자 이번 주4일...,-0.714814,0.785186,neg,pos,3,중간,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39905,2022-08-22,"코스피 장중 2,450대로 추락환율, 급등하며 1,340원 근접종합서울연합뉴스 이미...",0.26,"지수는 전장보다 25. 이날 서울 외환시장에서 원달러 환율은 장중 1,330원을 ...",1729,86,2,1.768957e-07,1.000000,"코스피 장중 2,450대로 추락환율, 급등하며 1,340원 근접종합서울연합뉴스 이미...",-1.000000,0.500000,neg,pos,0,하락,1
39906,2022-08-22,기아의 유럽 전략형 모델 씨드서울연합뉴스 기아의 유럽 맞춤형 전략 차종인 씨드eed...,0.26,기아의 유럽 전략형 모델 씨드서울연합뉴스 기아의 유럽 맞춤형 전략 차종인 씨드eed...,89,54,2,5.642329e-01,0.435767,기아의 유럽 전략형 모델 씨드서울연합뉴스 기아의 유럽 맞춤형 전략 차종인 씨드eed...,0.128466,1.628466,pos,pos,1,상승,3
39907,2022-08-22,"현대차그룹엔솔 인니 배터리셀 합작공장, 약 1조원 투자금 확보서울연합뉴스 최평천 기...",0.26,"현대차그룹엔솔 인니 배터리셀 합작공장, 약 1조원 투자금 확보서울연합뉴스 최평천 기...",907,456,2,1.326228e-07,1.000000,"현대차그룹엔솔 인니 배터리셀 합작공장, 약 1조원 투자금 확보서울연합뉴스 최평천 기...",-1.000000,0.500000,neg,pos,0,하락,1
39908,2022-08-22,"미, 탈레반과 아프간 동결 자산 해제 협상 계속 추진서울연합뉴스 김정은 기자 미국...",0.26,미국은 탈레반이 지난해 8월 친미 아프간 정부를 무너뜨리고 아프간을 장악하자 미국...,983,376,2,1.319802e-05,0.999987,"미, 탈레반과 아프간 동결 자산 해제 협상 계속 추진서울연합뉴스 김정은 기자 미국...",-0.999974,0.500026,neg,pos,0,하락,1


In [ ]:
df = df.rename(columns={'date_cate' : 'date'})
df.head()

,date,content,전일비_비율,text_rank_content,len,text_rank_len,target,mpc_pos,mpc_neg,mpck_token,score_diff,score_diff+tar,pos_neg,pos_neg_tardiff,kmm_label,kmm_label_n5,kmm_label_n_n5
0,2017-12-04,현대기아차 북미시장 고전 지속코나스팅어 기대로스앤젤레스연합뉴스 옥철 특파원 현대차...,-0.15,"현대차는 오토쇼에 데뷔한 콤팩트 스포츠유틸리티차 코나ona, 기아차는 2018 ...",958,288,2,0.996044,0.003956,현대기아차 북미시장 고전 지속코나스팅어 기대로스앤젤레스연합뉴스 옥철 특파원 현대차...,0.992088,2.492088,pos,pos,2,매우상승,4
1,2017-12-04,기업공시12월 1일서울경제 대표이사 손봉영에서 김종호로 변경 인터플렉스 유상증자에...,-0.15,기업공시12월 1일서울경제 대표이사 손봉영에서 김종호로 변경 인터플렉스 유상증자에...,1372,473,2,0.000003,0.999997,기업공시12월 1일서울경제 대표이사 손봉영에서 김종호로 변경 인터플렉스 유상증자에...,-0.999994,0.500006,neg,pos,0,하락,1
2,2017-12-04,뒷좌석 어린이 안전도 평가카시트 장착한 6세 아동 충격 더 크다헤럴드경제박도제 기자...,-0.15,뒷좌석 어린이 안전도 평가카시트 장착한 6세 아동 충격 더 크다헤럴드경제박도제 기자...,1618,392,2,0.000013,0.999987,뒷좌석 어린이 안전도 평가카시트 장착한 6세 아동 충격 더 크다헤럴드경제박도제 기자...,-0.999973,0.500027,neg,pos,0,하락,1
3,2017-12-04,"국산차, 연말 파격 할인싼타페 230만원캡티바 500만원이데일리 김보경 기자 국내 ...",-0.15,한국은 12월 한 달 동안 전 차종에 대해 515의 할인을 적용한다. 주력 차...,1014,237,2,0.000035,0.999965,"국산차, 연말 파격 할인싼타페 230만원캡티바 500만원이데일리 김보경 기자 국내 ...",-0.999930,0.500070,neg,pos,0,하락,1
4,2017-12-04,상장 대목이번주 진에어 등 7개 증시 입성서울연합인포맥스 김경림 기자 이번 주4일...,-0.15,진에어는 공모청약 경쟁률 134. 동양피스톤은 내연기관용 피스톤을 만드는 회사로 ...,722,133,2,0.142593,0.857407,상장 대목이번주 진에어 등 7개 증시 입성서울연합인포맥스 김경림 기자 이번 주4일...,-0.714814,0.785186,neg,pos,3,중간,2


In [ ]:
# 날짜 긍부정 targetpm 라벨링
df_pm_dict = {}
df_pm = pd.read_csv('/content/drive/MyDrive/뉴스원인분석_modeling/data/kia_new_label_sentsplit_news_label_1231.csv')
df_date = df_pm.groupby('date')[['targetpm']].mean()
for idx in df_date.index :
  # str_idx = ''.join(idx.split('-'))
  df_pm_dict[idx] = int(df_date.loc[idx, 'targetpm'])
df2 = df.copy()
df2['date'] = df2['date'].astype('str')
df2 = df2[df2['date'].isin(list(df_pm_dict.keys()))]
df2 = df2.reset_index(drop=True)
df2['targetpm'] = df2['date'].apply(lambda x : df_pm_dict[x])
df2.head(20)

,date,content,전일비_비율,text_rank_content,len,text_rank_len,target,mpc_pos,mpc_neg,mpck_token,score_diff,score_diff+tar,pos_neg,pos_neg_tardiff,kmm_label,kmm_label_n5,kmm_label_n_n5,targetpm
0,2017-12-04,현대기아차 북미시장 고전 지속코나스팅어 기대로스앤젤레스연합뉴스 옥철 특파원 현대차...,-0.15,"현대차는 오토쇼에 데뷔한 콤팩트 스포츠유틸리티차 코나ona, 기아차는 2018 ...",958,288,2,9.960439e-01,0.003956,현대기아차 북미시장 고전 지속코나스팅어 기대로스앤젤레스연합뉴스 옥철 특파원 현대차...,0.992088,2.492088,pos,pos,2,매우상승,4,1
1,2017-12-04,기업공시12월 1일서울경제 대표이사 손봉영에서 김종호로 변경 인터플렉스 유상증자에...,-0.15,기업공시12월 1일서울경제 대표이사 손봉영에서 김종호로 변경 인터플렉스 유상증자에...,1372,473,2,3.168241e-06,0.999997,기업공시12월 1일서울경제 대표이사 손봉영에서 김종호로 변경 인터플렉스 유상증자에...,-0.999994,0.500006,neg,pos,0,하락,1,1
2,2017-12-04,뒷좌석 어린이 안전도 평가카시트 장착한 6세 아동 충격 더 크다헤럴드경제박도제 기자...,-0.15,뒷좌석 어린이 안전도 평가카시트 장착한 6세 아동 충격 더 크다헤럴드경제박도제 기자...,1618,392,2,1.344403e-05,0.999987,뒷좌석 어린이 안전도 평가카시트 장착한 6세 아동 충격 더 크다헤럴드경제박도제 기자...,-0.999973,0.500027,neg,pos,0,하락,1,1
3,2017-12-04,"국산차, 연말 파격 할인싼타페 230만원캡티바 500만원이데일리 김보경 기자 국내 ...",-0.15,한국은 12월 한 달 동안 전 차종에 대해 515의 할인을 적용한다. 주력 차...,1014,237,2,3.493616e-05,0.999965,"국산차, 연말 파격 할인싼타페 230만원캡티바 500만원이데일리 김보경 기자 국내 ...",-0.999930,0.500070,neg,pos,0,하락,1,1
4,2017-12-04,상장 대목이번주 진에어 등 7개 증시 입성서울연합인포맥스 김경림 기자 이번 주4일...,-0.15,진에어는 공모청약 경쟁률 134. 동양피스톤은 내연기관용 피스톤을 만드는 회사로 ...,722,133,2,1.425929e-01,0.857407,상장 대목이번주 진에어 등 7개 증시 입성서울연합인포맥스 김경림 기자 이번 주4일...,-0.714814,0.785186,neg,pos,3,중간,2,1
5,2017-12-04,현대기아차 북미시장 고전 지속코나스팅어 기대11월에도 현대차와 기아차가 북미 시장에...,-0.15,현대기아차 북미시장 고전 지속코나스팅어 기대11월에도 현대차와 기아차가 북미 시장에...,789,299,2,9.996573e-01,0.000343,현대기아차 북미시장 고전 지속코나스팅어 기대11월에도 현대차와 기아차가 북미 시장에...,0.999315,2.499315,pos,pos,2,매우상승,4,1
6,2017-12-04,"현대기아차, 11월 북미판매 동반부진신차로 활로이데일리 노재웅 기자 현대기아자동차가...",-0.15,"현대기아차, 11월 북미판매 동반부진신차로 활로이데일리 노재웅 기자 현대기아자동차가...",1069,274,2,9.965403e-01,0.003460,"현대기아차, 11월 북미판매 동반부진신차로 활로이데일리 노재웅 기자 현대기아자동차가...",0.993081,2.493081,pos,pos,2,매우상승,4,1
7,2017-12-04,"현대기아차, 올해 내수 목표치 가시권나머지는 물 건너가이데일리 노재웅 기자 한 해를...",-0.15,이달 비슷한 수준의 판매 실적을 올린다면 약 69만3000대로 올해 목표량인 68...,2299,209,2,1.098262e-03,0.998902,"현대기아차, 올해 내수 목표치 가시권나머지는 물 건너가이데일리 노재웅 기자 한 해를...",-0.997803,0.502197,neg,pos,0,하락,1,1
8,2017-12-04,일자리 창출 충북에서 배워라충북도내 일자리 증가율 1위 제천시 경쟁력은제천이데일리 ...,-0.15,일자리 창출 충북에서 배워라충북도내 일자리 증가율 1위 제천시 경쟁력은제천이데일리 ...,1851,387,2,9.797455e-01,0.020255,일자리 창출 충북에서 배워라충북도내 일자리 증가율 1위 제천시 경쟁력은제천이데일리 ...,0.959491,2.459491,pos,pos,2,매우상승,4,1
9,2017-12-04,"기아차, 2018년형 7 출시부분 자율주행 기술 탑재이데일리 노재웅 기자 기아자동차...",-0.15,"기아차, 2018년형 7 출시부분 자율주행 기술 탑재이데일리 노재웅 기자 기아자동차...",1108,252,2,4.111699e-06,0.999996,"기아차, 2018년형 7 출시부분 자율주행 기술 탑재이데일리 노재웅 기자 기아자동차...",-0.999992,0.500008,neg,pos,0,하락,1,1


In [ ]:
df_gr = df2.groupby('date')[['kmm_label_n_n5', 'targetpm']].mean()
# 날짜별로 train_test_split
train_date, test_date = train_test_split(df_gr, test_size=0.1, random_state=2022)
train_date_list = train_date.index
test_date_list = test_date.index
train_data = df2[df2['date'].isin(train_date_list)][['mpck_token', 'kmm_label_n_n5', 'target', 'date','targetpm']]
test_data = df2[df2['date'].isin(test_date_list)][['mpck_token', 'kmm_label_n_n5', 'target' ,'date','targetpm']]
train_data.shape, test_data.shape

((35897, 5), (3933, 5))

In [ ]:
df['target'].value_counts()

1    14636
3    13858
2     9484
4     1434
0      498
Name: target, dtype: int64

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
max_len = 512 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 16
warmup_ratio = 0.1
num_epochs = 50
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 5, # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
bertmodel2= torch.load("/content/drive/MyDrive/뉴스원인분석_modeling/model/model_1226_KoBERT_md2_m128_ep50_kss_1228.pt")
bertmodel2.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [ ]:
test_data

,mpck_token,kmm_label_n_n5,target,date,targetpm
175,한기평 내년 자동차업황 비우호적미중 시장 주목서울연합인포맥스 이민재 기자 국내 자...,2,3,2017-12-12,3
176,"이브자리, 연말 사랑의 이불 1000채 기부이데일리 박경훈 기자 침구전문업체 이브자...",2,3,2017-12-12,3
192,"단독 이노션, 광고대행사 인수 잰걸음현대자동차그룹 계열 광고대행사인 이노션이 기아...",4,3,2017-12-12,3
197,"김승연, 한화큐셀 공장 방문세계 1등 지위 강화치둥공장, 미래 태양광 사업 이끄는...",2,3,2017-12-12,3
198,광주 서구의회 기아차 공장 주변 악취 원인 규명해야광주연합뉴스 장아름 기자 광주 ...,4,3,2017-12-12,3
...,...,...,...,...,...
39588,세련되게 잘 빠졌네 [SEP] 패밀리카 뺨치는 가성비쌍용자동차의 토레스는 지난해 렌...,4,2,2022-08-09,3
39590,엄마 마음 훔치더니.. [SEP] 당당하게 가족차 된 셀토스 시승기국내 소형 스포츠...,4,2,2022-08-09,3
39591,"서머 정규 리그 막바지젠지, 1위 확정까지 단 1승리그오브레전드챔피언스코리아대표 오...",1,2,2022-08-09,3
39592,"지바이크, 성남 킥보드 인수전동킥보드 공유 서비스 플랫폼 지쿠터를 운영 중인 지바이...",1,2,2022-08-09,3


In [ ]:
# batch_size 수정 코드
batch_size = 128

test_data3 = test_data.copy()
test_data3 = test_data3.reset_index(drop=True )
test_data3['pred'] = ''

bertmodel2.eval()
unseen_values = test_data3[['mpck_token', 'kmm_label_n_n5']].values
test_set = BERTDataset(unseen_values, 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=5)
for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  out = bertmodel2(token_ids, valid_length, segment_ids)
  for outn in range(len(out)) :
    pred = int(out[outn].argmax().cpu().numpy())
    idx = batch_size * batch_id + outn
    test_data3.iloc[idx, -1] = pred
    if idx % 5000 == 0 :
      print(idx, pred)
test_data3.head(20)

0it [00:00, ?it/s]

0 2


,mpck_token,kmm_label_n_n5,target,date,targetpm,pred
0,한기평 내년 자동차업황 비우호적미중 시장 주목서울연합인포맥스 이민재 기자 국내 자...,2,3,2017-12-12,3,2
1,"이브자리, 연말 사랑의 이불 1000채 기부이데일리 박경훈 기자 침구전문업체 이브자...",2,3,2017-12-12,3,2
2,"단독 이노션, 광고대행사 인수 잰걸음현대자동차그룹 계열 광고대행사인 이노션이 기아...",4,3,2017-12-12,3,2
3,"김승연, 한화큐셀 공장 방문세계 1등 지위 강화치둥공장, 미래 태양광 사업 이끄는...",2,3,2017-12-12,3,2
4,광주 서구의회 기아차 공장 주변 악취 원인 규명해야광주연합뉴스 장아름 기자 광주 ...,4,3,2017-12-12,3,2
5,"자유로 연비 르노삼성 6 e, 자유로를 달리다이데일리 오토in 김학수 기자 르노삼성...",2,3,2017-12-12,3,2
6,"노블클라쎄, 합리적 프리미엄 내세운 고급 미니밴노블클라쎄 카니발 9 국내 유일의 프...",2,3,2017-12-12,3,2
7,한화큐셀 공장 찾은 김승연 태양광은 한화의 미래인수후 7년만에 치둥공장 첫 방문 글...,2,3,2017-12-12,3,2
8,"사드 피해 보듬고 기업 살리자..., 현대차 충칭공장 간다대통령 오늘 방중 16일 ...",2,3,2017-12-12,3,2
9,"시승기스팅어 3가지 매력매혹적 디자인주행성능, 그리고 가격까지헤럴드경제배두헌 기자 ...",2,3,2017-12-12,3,2


In [ ]:
# 뉴스별 라벨링
# 뉴스 빈도 + 날 빈도
test_data4 = test_data.copy()
test_data4['pred'] = test_data3['pred'].values 
test_data4['pred'] = test_data4['pred'].astype('int')

df_gr = test_data4.groupby('date')[['target', 'pred', 'targetpm']].mean()
df_date = df_gr.index
for idx in df_date:
  df_gr.loc[idx, 'cnt_pred'] = int(test_data4[test_data4['date'] == idx ]['pred'].value_counts().idxmax())

df_gr['pred_label'] = df_gr['cnt_pred'].apply(lambda x : int(np.round(x)))
df_gr['target'] = df_gr['target'].apply(lambda x : int(np.round(x)))
df_gr['pred_pm'] = df_gr['pred_label'].apply(lambda x: 1 if x< 2 else 3)


df_gr = df_gr.reset_index()
for idx in df_gr.index :
  if df_gr.loc[idx, 'target'] == df_gr.loc[idx, 'pred_label']:
    temp_eval = True
  else :
    temp_eval = False
  df_gr.loc[idx, 'eval_5'] = temp_eval

for idx in df_gr.index :
  if df_gr.loc[idx, 'target'] == 0 :
    df_gr.loc[idx, 'target_3label'] = 1
  elif df_gr.loc[idx, 'target'] == 4 :
    df_gr.loc[idx, 'target_3label'] = 3
  else :
    df_gr.loc[idx, 'target_3label'] = int(df_gr.loc[idx, 'target'])

  if df_gr.loc[idx, 'pred_label'] == 0 :
    df_gr.loc[idx, 'pred_label_3label'] = 1
  elif df_gr.loc[idx, 'pred_label'] == 4 :
    df_gr.loc[idx, 'pred_label_3label'] = 3
  else :
    df_gr.loc[idx, 'pred_label_3label'] = int(df_gr.loc[idx, 'pred_label'])

df_gr['pred_label_3label'] = df_gr['pred_label_3label'].apply(lambda x : int(x))

for idx in df_gr.index :
  if df_gr.loc[idx, 'target_3label'] == df_gr.loc[idx, 'pred_label_3label']:
    temp_eval = True
  else :
    temp_eval = False
  df_gr.loc[idx, 'eval_3'] = temp_eval

for idx in df_gr.index :
  if df_gr.loc[idx, 'targetpm'] == df_gr.loc[idx, 'pred_pm']:
    temp_eval = True
  else :
    temp_eval = False
  df_gr.loc[idx, 'eval_pm'] = temp_eval



print('5중분류 정확도 :', round(df_gr['eval_5'].sum() / len(df_gr), 2), '\n')
print('3중분류 정확도 :', round(df_gr['eval_3'].sum() / len(df_gr), 2), '\n')
print('이진분류 정확도 :', round(df_gr['eval_pm'].sum() / len(df_gr), 2), '\n')

df_gr.head()

5중분류 정확도 : 0.32 

3중분류 정확도 : 0.32 

이진분류 정확도 : 0.62 



,date,target,pred,targetpm,cnt_pred,pred_label,pred_pm,eval_5,target_3label,pred_label_3label,eval_3,eval_pm
0,2017-12-12,3,2.272727,3.0,2.0,2,3,False,3.0,2,False,True
1,2018-01-12,1,2.375000,1.0,2.0,2,3,False,1.0,2,False,False
2,2018-01-31,3,2.238095,3.0,2.0,2,3,False,3.0,2,False,True
3,2018-04-20,2,2.352941,3.0,2.0,2,3,True,2.0,2,True,True
4,2018-05-04,2,2.250000,1.0,2.0,2,3,True,2.0,2,True,False


In [ ]:
test_data3['pred'].value_counts()

2    2934
4     742
3     236
0      21
Name: pred, dtype: int64

In [ ]:
df_gr['pred_label'].value_counts()

2    117
Name: pred_label, dtype: int64

In [ ]:
df_gr['target'].value_counts()

3    43
2    38
1    29
4     6
0     1
Name: target, dtype: int64

In [ ]:
df_gr['cnt_pred'].mean()

2.0

In [ ]:
df_gr['eval_pm'].value_counts(normalize=True)

True     0.623932
False    0.376068
Name: eval_pm, dtype: float64

In [ ]:
df_gr[df_gr['eval_pm'] == True]['pred_label'].value_counts()

2    73
Name: pred_label, dtype: int64